In [3]:
import json
import requests

Tworzenie obiektu json w pamięci

In [4]:
data = {
    "car": {
        "mark": "Toyota",
        "model": "Supra",
        "color": "Black"
    }
}

Zapisywanie obiektu do pliku, metoda dump()

In [5]:
with open("json/first_saved_data.json", "w") as file_to_write:
    json.dump(data, file_to_write)

Zapis do natywnego stringa pythonowskiego

In [6]:
json_string = json.dumps(data)
json_string

'{"car": {"mark": "Toyota", "model": "Supra", "color": "Black"}}'

Użycie parametru indent w dumps()

In [7]:
json_indent=json.dumps(data, indent=4)
json_indent

'{\n    "car": {\n        "mark": "Toyota",\n        "model": "Supra",\n        "color": "Black"\n    }\n}'

Enkodowanie i dekodowanie obiektu

In [9]:
money = (10, "$")
encoded_money = json.dumps(money)
decoded_money = json.loads(encoded_money)

print(money == decoded_money)
print(type(money))
print(type(decoded_money))
print(money == tuple(decoded_money))

False
<class 'tuple'>
<class 'list'>
True


### Przykład deserializacji

Deklaracja obiektu json

string

In [12]:
json_string = """
{
    "knight": {
        "name": "Markus the Huge",
        "weapon": "Shield",
        "relatives": [
            {
                "name": "Mark the Brave",
                "type": "Brother"
            }
        ]
    }
}
"""
data = json.loads(json_string)
data

{'knight': {'name': 'Markus the Huge',
  'weapon': 'Shield',
  'relatives': [{'name': 'Mark the Brave', 'type': 'Brother'}]}}

plik

In [13]:
with open("json/first_saved_data.json", "r") as read_file:
    data = json.load(read_file)
data

{'car': {'mark': 'Toyota', 'model': 'Supra', 'color': 'Black'}}

Tworzenie zapytania (request)

In [14]:
response = requests.get("https://jsonplaceholder.typicode.com/posts")
posts = json.loads(response.text)

print(response.json()==posts)
print(type(posts))
print(posts[:10])

True
<class 'list'>
[{'userId': 1, 'id': 1, 'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit', 'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'}, {'userId': 1, 'id': 2, 'title': 'qui est esse', 'body': 'est rerum tempore vitae\nsequi sint nihil reprehenderit dolor beatae ea dolores neque\nfugiat blanditiis voluptate porro vel nihil molestiae ut reiciendis\nqui aperiam non debitis possimus qui neque nisi nulla'}, {'userId': 1, 'id': 3, 'title': 'ea molestias quasi exercitationem repellat qui ipsa sit aut', 'body': 'et iusto sed quo iure\nvoluptatem occaecati omnis eligendi aut ad\nvoluptatem doloribus vel accusantium quis pariatur\nmolestiae porro eius odio et labore et velit aut'}, {'userId': 1, 'id': 4, 'title': 'eum et est occaecati', 'body': 'ullam et saepe reiciendis voluptatem adipisci\nsit amet autem assumenda provident rer

Test na szybkosc metody POST

In [19]:
posts_by_user={}
for post in posts:
    try:
        if len(post["body"])> posts_by_user[post["userId"]]:
            posts_by_user[post["userId"]]=len(post["body"])
    except KeyError:
            posts_by_user[post["userId"]]=len(post["body"])

longest_post = sorted(posts_by_user.items(),
                      key=lambda x:x[1], reverse=True)
print(posts_by_user)
print(longest_post)

users_longest_posts=[]
for longest in longest_post:
    if longest_post[0][1]>longest[1]:
        break
    users_longest_posts.append(str(longest[0]))

user_string = " and ".join(users_longest_posts)
s = "s" if len(users_longest_posts)>1 else ""
print(f'Uzytkownik{s} {user_string} wygrywa z iloscia {longest_post[0][1]} znakow')

{1: 206, 2: 225, 3: 190, 4: 212, 5: 193, 6: 197, 7: 218, 8: 196, 9: 198, 10: 215}
[(2, 225), (7, 218), (10, 215), (4, 212), (1, 206), (9, 198), (6, 197), (8, 196), (5, 193), (3, 190)]
Uzytkownik 2 wygrywa z iloscia 225 znakow


Filtrowanie 5 użytkownikow którzy zapisali najwiecej postów i zapis do pliku json

In [21]:
users = longest_post[:5]

def keep(post):
    counter = False
    for user in users:
        if user[1] == len(post["body"]) and post["userId"] == user[0]:
            counter = True
    return counter

with open("json/second_saved_data_filter.json","w") as data_file:
    filtered_posts = list(filter(keep,posts))
    json.dump(filtered_posts,data_file,indent=2)

radzenie sobie z typem "complex"

In [22]:
def encode_complex(z):
    if isinstance(z,complex):
        return (z.real, z.imag)
    else:
        type_name = z.__class__.__name__
        raise TypeError(f"Object of type '{type_name}' is not JSON serializable")

json.dumps(9+5j,default=encode_complex)

'[9.0, 5.0]'

JSONEncoder, nadpisanie metody default poprzez stworzenie klasy podrzędnej

In [24]:
class ComplexEncoder(json.JSONEncoder):
    def default(self, z):
        if isinstance(z, complex):
            return (z.real, z.imag)
        else:
            return super().default(z)

print(json.dumps(7 + 2j, cls=ComplexEncoder))
encode = ComplexEncoder()
encode.encode(5 + 55j)

[7.0, 2.0]


'[5.0, 55.0]'

Dekodowanie niestandardowych obiektow z pliku

In [28]:
def decode_complex(dct):
    if "__complex__" in dct:
        return complex(dct["real"], dct["imag"])
    return dct

with open("json/complex_data.json") as complex_data:
     data = complex_data.read()
     numbers = json.loads(data, object_hook=decode_complex)
print(numbers)



[(42+36j), (64+11j)]
